# Annulus: different distributions
* Let `P` and `Q` be two point cloud sampled from two distinct distributions on an annulus.
* This notebook illustrates the problem of constructing persistence modules $$VR(P) \rightarrow VR(P \cup Q) \leftarrow VR(Q)$$
* This notebook also implements the similarity-centric analogous bars method.

In [1]:
include("../../../extension_method.jl")

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-8144943044791845393\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-2627170538794260749\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-8144943044791845393\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

┌ Warning: ORCA.jl has been deprecated and all savefig functionality
│ has been implemented directly in PlotlyBase itself.
│ 
│ By implementing in PlotlyBase.jl, the savefig routines are automatically
│ available to PlotlyJS.jl also.
└ @ ORCA /opt/julia/packages/ORCA/U5XaN/src/ORCA.jl:8


Main.ext

In [2]:
using .ext
using Distances
using Distributions
using Eirene
using Plots
using JLD
using Measures

# 1. Load points

In [3]:
# load points
results = load("two_circles.jld")
points = results["points"]
n_small = results["n_small"]
n_large = results["n_large"]

P = points[1:n_small,:] # points from small circle
Q = points[n_small+1:end, :]; # points from large circle

┌ Warning: JLD incorrectly extends FileIO functions (see FileIO documentation)
└ @ FileIO /opt/julia/packages/FileIO/JA3Vl/src/loadsave.jl:215


Note: The points were generated using the following code

In [ ]:
"""
# generate small circle
n_small = 100
theta = rand(Uniform(0,2*pi), n_small)
distance = rand(Normal(1, 0.2), n_small);

small = zeros(n_small,2)
for i=1:n_small
    small[i, 1] = distance[i] * cos(theta[i]) 
    small[i, 2] = distance[i] * sin(theta[i])
end

# generate large circle
n_large = 20
theta = rand(Uniform(0,2*pi), n_large)
distance = rand(Normal(2, 0.2), n_large);

large = zeros(n_large,2)
for i=1:n_large
    large[i, 1] = distance[i] * cos(theta[i]) 
    large[i, 2] = distance[i] * sin(theta[i])
end
"""

In [4]:
plot_P_Q(P, Q)

<!DOCTYPE html>
 
 
 Plots.jl

# 2. Plot barcodes on $P$, $Q$, and $P \cup Q$

In [7]:
# compute distances among points
P_all = vcat(P, Q)
D = pairwise(Euclidean(), transpose(P_all), transpose(P_all), dims=2)

# Define submatrices 
D_P = D[1:n_small, 1:n_small]
D_Q = D[n_small+1:end, n_small+1:end]
D_P_Q = D[1:n_small, n_small+1:end]
    # rows (landmarks): P
    # columns (witness) : Q
D_Q_P = D[n_small+1:end, 1:n_small];
    # rows (landmarks): Q
    # columns (witness) : P

In [8]:
# run persistence
C_P = eirene(D_P, record = "all")
C_union = eirene(D, record = "all")
C_Q = eirene(D_Q, record = "all");

In [9]:
# plot all barcodes in dim 1
barcode_P = barcode(C_P, dim = 1)
barcode_union = barcode(C_union, dim = 1)
barcode_Q = barcode(C_Q, dim = 1)

l = grid(3,1)
p1 = plot_barcode(barcode_P, lw = 3, title = "Barcode(VR(P))", titlefontsize = 12)
p2 = plot_barcode(barcode_union, lw = 3, title = "Barcode(VR(P U Q))", titlefontsize = 12)
p3 = plot_barcode(barcode_Q, lw = 3, title = "Barcode(VR(Q))", titlefontsize = 12)
plot(p1, p2, p3, layout = l, size = (500, 700))

<!DOCTYPE html>
 
 
 Plots.jl

The features of P and Q exist at very different scales. Therefore, any attempt to matching between `barcode(VR(Q))` and `barcode(VR(P U Q))` will be trivial.

# 3. Implement the similarity-centric analogous bars method

In [10]:
# Compute Vietoris-Rips persistence 
dim = 1
VR_P = eirene(D_P, record = "all", maxdim = dim)
VR_Q = eirene(D_Q, record = "all", maxdim = dim )

# Compute Witness filtration
W_P = compute_Witness_persistence(D_P_Q, maxdim = dim)
W_Q = compute_Witness_persistence(D_Q_P, maxdim = dim);

In [11]:
# plot all four barcodes
barcode_P = barcode(VR_P, dim = 1)
barcode_W_PQ = barcode(W_P["eirene_output"], dim = 1)
barcode_W_QP = barcode(W_Q["eirene_output"], dim = 1)
barcode_Q = barcode(VR_Q, dim = 1)

l = grid(4,1)
p1 = plot_barcode(barcode_P, lw = 3, title = "Barcode(VR(P))", titlefontsize = 12)
p2 = plot_barcode(barcode_W_PQ, lw = 3, title = "Barcode(W(P,Q))", titlefontsize = 12)
p3 = plot_barcode(barcode_W_QP, lw = 3, title = "Barcode(W(Q,P))", titlefontsize = 12)
p4 = plot_barcode(barcode_Q, lw = 2, title = "Barcode(VR(Q))", titlefontsize = 12)
plot(p1, p2, p3, p4, layout = l, size = (500, 700))

<!DOCTYPE html>
 
 
 Plots.jl

In [12]:
# select Witness bar
W_PQ_bar = 1

# run extension
extension_P, extension_Q = run_similarity_analogous(VR_P = VR_P, 
                                                    D_P = D_P, 
                                                    VR_Q = VR_Q, 
                                                    D_Q = D_Q, 
                                                    W_PQ = W_P, 
                                                    W_PQ_bar = W_PQ_bar, 
                                                    dim = dim);

In [13]:
plot_analogous_bars(extension_P, extension_Q, lw_VR_P = 8)

<!DOCTYPE html>
 
 
 Plots.jl